# Run SuSiE Run!
## Data processing for fine-mapping with SusiE and DAP-G
This is an informative Jupyter notebook that was written to help pull and process data for analysis using the fine-mapping programs SuSiE and DAP-G. These are main goals of this script:
1. Pull GWAS summary stats and reference panel data
2. Make a LD (linkage disequilibrium) matrix with the reference panel data
3. Align and clean the summary stats and LD matrix
4. Fine-map the region using 2 different programs:
    - A program written in R named SuSiE
    - A program written in C++ named DAP-G

## 1. Pulling GWAS summary statistics and reference panel data
*Important Note: these instructions are specific to data that **is not** publicly accessible* 

1. If you don't have biowulf access, [request it](https://hpc.nih.gov/docs/accounts.html)
2. Request access to data files (ask your PI/supervisor)
3. [Connect to Biowulf](https://hpc.nih.gov/docs/connect.html) 

Once you're connected, edit the file **scripts/config.sh** to match your region of interest. Don't forget to include the path to the project directory so that the files don't pile up in shared folders.

In [ ]:
# Once your config is set up and saved, run the following script
!sh scripts/data.sh

In [ ]:
# Check that your files are present, there should be a bim/fam/bed file with the rsID in your config file and a txt file with the chr number
!source scripts/config.sh
!ls $dir